In [29]:
!pip install pretty_midi

     |████████████████████████████████| 5.6 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 59 kB/s s eta 0:00:01
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-py3-none-any.whl size=5591952 sha256=6ae2e0c9ac02a53188789f433a81a99b309fba3c1ac53900ed0acbb47e3224c2
  Stored in directory: /home/naorko/.cache/pip/wheels/2a/5a/e3/30eeb9a99350f3f7e21258fcb132743eef1a4f49b3505e76b6
Successfully built pretty-midi


In [30]:
import pretty_midi
import pandas as pd
import numpy as np

In [27]:
cols = ['Singer', 'Song Name', 'Lyrics']
df_train = pd.read_csv('datasets/lyrics_train_set.csv', names=cols)
df_test = pd.read_csv('datasets/lyrics_test_set.csv', names=cols)

In [25]:
df_train

,Singer,Song Name,Lyrics
0,elton john,candle in the wind,goodbye norma jean & though i never knew you a...
1,gerry rafferty,baker street,winding your way down on baker street & lite i...
2,gerry rafferty,right down the line,you know i need your love & you've got that ho...
3,2 unlimited,tribal dance,come on check it out ya'll & (come on come on!...
4,2 unlimited,let the beat control your body,let the beat control your body & let the beat ...
...,...,...,...
610,don henley,dirty laundry,i make my living off the evening news & just g...
611,don henley,new york minute,harry got up & dressed all in black & went dow...
612,bob dylan,subterranean homesick blues,johnny's in the basement & mixing up the medic...
613,goldfinger,mable,i met her sunday that was yesterday & the girl...


In [28]:
df_test

,Singer,Song Name,Lyrics
0,the bangles,eternal flame,close your eyes give me your hand darling & d...
1,billy joel,honesty,if you search for tenderness & it isn't hard ...
2,cardigans,lovefool,dear i fear we're facing a problem & you love...
3,aqua,barbie girl,hiya barbie & hi ken! & do you want to go for...
4,blink 182,all the small things,all the small things & true care truth brings...


In [34]:
pm = pretty_midi.PrettyMIDI('datasets/midi_files/aladdin_-_A_whole_new_world.mid')
pm

In [38]:
print('There are {} time signature changes'.format(len(pm.time_signature_changes)))
print('There are {} instruments'.format(len(pm.instruments)))
print('Instrument 3 has {} notes'.format(len(pm.instruments[0].notes)))
print('Instrument 4 has {} pitch bends'.format(len(pm.instruments[4].pitch_bends)))
print('Instrument 5 has {} control changes'.format(len(pm.instruments[5].control_changes)))

There are 1 time signature changes
There are 9 instruments
Instrument 3 has 227 notes
Instrument 4 has 0 pitch bends
Instrument 5 has 0 control changes
